In [14]:
import json
import os
import pandas as pd
import matplotlib.pyplot as plt

In [15]:
jsons = []

In [16]:
for file in os.listdir("data/matches/raw/"):
    data = json.load(open("data/matches/raw/"+file))
    jsons.append(data)

In [23]:
data = jsons[0]

In [24]:
data.keys()

dict_keys(['id', 'name', 'NameWithoutExtension', 'date', 'source', 'comment', 'status', 'client_name', 'hostname', 'type', 'tickrate', 'server_tickrate', 'duration', 'ticks', 'map_name', 'path', 'cheater_count', 'score_team1', 'score_team2', 'score_half1_team1', 'score_half1_team2', 'score_half2_team1', 'score_half2_team2', '5k_count', '4k_count', '3k_count', '2k_count', '1k_count', 'team_ct', 'team_t', 'team_surrender', 'team_winner', 'rounds', 'players', 'most_killing_weapon', 'overtimes', 'most_headshot_player', 'most_bomb_planted_player', 'most_entry_kill', 'bomb_planted', 'bomb_defused', 'bomb_exploded', 'kills', 'weapon_fired', 'player_blinded_events', 'player_hurted', 'kill_count', 'clutch_count', 'trade_kill_count', 'bomb_defused_count', 'bomb_planted_count', 'bomb_exploded_count', 'flashbang_thrown_count', 'smoke_thrown_count', 'he_thrown_count', 'decoy_thrown_count', 'molotov_thrown_count', 'incendiary_thrown_count', 'decoys', 'incendiaries', 'molotovs', 'damage_health_count'

In [72]:
pd.DataFrame.from_records(data['bomb_defused'])

,defuser_steamid,defuser_name,site,tick,seconds
0,76561198164970560,yuurih,A,174469,1361.90173
1,76561198988083423,honda,B,204142,1593.77734
2,76561198156353883,CeRq,B,276883,2162.67578
3,76561197978481975,tarik,A,366142,2860.45679


In [68]:
x

,score,score_first_half,score_second_half,team_name,team_players
0,9,4,5,Evil Geniuses,"{'steamid': '76561198988083423', 'name': 'hond..."
1,9,4,5,Evil Geniuses,"{'steamid': '76561197996370184', 'name': 'VINI..."
2,9,4,5,Evil Geniuses,"{'steamid': '76561198058500492', 'name': 'KSCE..."
3,9,4,5,Evil Geniuses,"{'steamid': '76561198043769572', 'name': 'arT'..."
4,9,4,5,Evil Geniuses,"{'steamid': '76561198164970560', 'name': 'yuur..."


In [44]:
pd.json_normalize(x[x.columns[-1]]).columns[:20]

Index(['kd', 'mvp_count', 'hltv_rating', 'esea_rws', 'shot_count', 'hit_count',
       'is_vac_banned', 'is_ow_banned', 'flashbang_count', 'smoke_count'],
      dtype='object')